# Introduction/Business Problem

Pizza is arguably one of the world's most popular foods today, especially in the United States where pizza restaurant sales topped $46 billion dollars in 2019. [https://www.statista.com/statistics/291403/us-pizza-restaurant-sales/] About 1 in 8 Americans consume pizza on any given day. [https://www.ars.usda.gov/ARSUserFiles/80400530/pdf/DBrief/11_consumption_of_pizza_0710.pdf] This highly-craved food traces its roots back to antiqity, but was not introduced to the U.S. until the arrival of Italian immigrants in the 1800's, where it was popular with that population, in among other areas, New York City. However, it didn't gain the popularity it enjoys today until U.S. troops returned from Italy in World War II, craving the cuisine they were introduced to over there. Given its prevelance, there are more options for the pizza connoisseur then ever before. Despite the saturation, there are seemingly limitless opportunities to get in on the action yourself given its popularity.
On your first trip to New York City you fell in love with Manhattan, and have always dreamed of opening your own Pizza Place there. With continuing growth in both business and lesiure travel, [https://indd.adobe.com/view/fcc4cd9f-7386-4b52-a39b-c401266a137f] you feel now is the time to take action.

So where in Manhattan should you aim to open your pizza place, and are there any highly rated locations in the area you could emulate?

# Data

To solve this delimma, we'll need New York City neghborhood data including boroughs, longitude, latitude, and restaurant ratings.

https://cocl.us/new_york_dataset will provide us New York City data containing the neighborhoods and boroughs, latitudes, and longitudes.

All data related to locations and quality of pizza places will be obtained via the FourSquare API utilized via the Request library in Python.

# Method

Data will be collected and processed into dataframes.

Folium will be utilized to visualize and map locations.

FourSquare will be utilized to locate and explore venues.

One Hot encoding will determine the frequency of pizza places per neighborhood.

# Define Variables

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


# Now let's begin to explore Manhattan

In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [12]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


# Map out the neighborhoods in Manhattan

In [13]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

# Define credentials

In [75]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = ''
# LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [15]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [16]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=W03DEFIYSYUW3GCTD0ZGYPKADF0Q1DDPJYL1BUQ4S4I2TIP3&client_secret=TF0ZKC1WEP2XAUH3TO2PYBTKEKSW5DNZV0LRZHTOW0I1UKF3&v=20200926&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f7089adf6f5be02d563a3a6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [24]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3219, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [25]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,66,66,66,66,66,66
Carnegie Hill,90,90,90,90,90,90
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,40,40,40,40,40,40
East Village,100,100,100,100,100,100


In [26]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 325 uniques categories.


# Now let's use one hot encoding to explore Manhattan venues and look for Pizza places

In [27]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Venue Category'] = manhattan_venues['Venue Category'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head(100)

,Venue Category,Battery Park City,Carnegie Hill,Central Harlem,Chelsea,Chinatown,Civic Center,Clinton,East Harlem,East Village,Financial District,Flatiron,Gramercy,Greenwich Village,Hamilton Heights,Hudson Yards,Inwood,Lenox Hill,Lincoln Square,Little Italy,Lower East Side,Manhattan Valley,Manhattanville,Marble Hill,Midtown,Midtown South,Morningside Heights,Murray Hill,Noho,Roosevelt Island,Soho,Stuyvesant Town,Sutton Place,Tribeca,Tudor City,Turtle Bay,Upper East Side,Upper West Side,Washington Heights,West Village,Yorkville
0,Pizza Place,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Yoga Studio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Diner,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Coffee Shop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Donut Shop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Gym,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Gym,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Department Store,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Coffee Shop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Pharmacy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
manhattan_onehot.shape

(3219, 41)

In [29]:
manhattan_grouped = manhattan_onehot.groupby('Venue Category').mean().reset_index()
manhattan_grouped

,Venue Category,Battery Park City,Carnegie Hill,Central Harlem,Chelsea,Chinatown,Civic Center,Clinton,East Harlem,East Village,Financial District,Flatiron,Gramercy,Greenwich Village,Hamilton Heights,Hudson Yards,Inwood,Lenox Hill,Lincoln Square,Little Italy,Lower East Side,Manhattan Valley,Manhattanville,Marble Hill,Midtown,Midtown South,Morningside Heights,Murray Hill,Noho,Roosevelt Island,Soho,Stuyvesant Town,Sutton Place,Tribeca,Tudor City,Turtle Bay,Upper East Side,Upper West Side,Washington Heights,West Village,Yorkville
0,Accessories Store,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000
1,Adult Boutique,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,African Restaurant,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,American Restaurant,0.000000,0.000000,0.025974,0.051948,0.051948,0.038961,0.064935,0.000000,0.025974,0.038961,0.038961,0.051948,0.025974,0.000000,0.038961,0.025974,0.000000,0.038961,0.000000,0.012987,0.012987,0.012987,0.000000,0.012987,0.038961,0.038961,0.051948,0.012987,0.000000,0.012987,0.000000,0.025974,0.064935,0.012987,0.025974,0.025974,0.025974,0.012987,0.077922,0.000000
4,Antique Shop,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Arepa Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000
6,Argentinian Restaurant,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Art Gallery,0.000000,0.000000,0.038462,0.153846,0.000000,0.000000,0.038462,0.000000,0.038462,0.000000,0.076923,0.038462,0.038462,0.000000,0.038462,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.153846,0.000000,0.038462,0.000000,0.000000,0.076923,0.000000,0.000000,0.038462,0.000000,0.000000,0.038462,0.000000
8,Art Museum,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000
9,Arts & Crafts Store,0.000000,0.000000,0.000000,0.000000,0.00

In [30]:
manhattan_grouped.shape

(325, 41)

In [31]:
num_top_venues = 20

for hood in manhattan_grouped['Venue Category']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Venue Category'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Accessories Store----
                  venue  freq
0          West Village  0.33
1    Washington Heights  0.33
2       Upper West Side  0.33
3     Battery Park City  0.00
4      Roosevelt Island  0.00
5           Marble Hill  0.00
6               Midtown  0.00
7         Midtown South  0.00
8   Morningside Heights  0.00
9           Murray Hill  0.00
10                 Noho  0.00
11      Stuyvesant Town  0.00
12                 Soho  0.00
13        Carnegie Hill  0.00
14         Sutton Place  0.00
15              Tribeca  0.00
16           Tudor City  0.00
17           Turtle Bay  0.00
18      Upper East Side  0.00
19       Manhattanville  0.00


----Adult Boutique----
                  venue  freq
0          Sutton Place   1.0
1     Battery Park City   0.0
2                  Soho   0.0
3           Marble Hill   0.0
4               Midtown   0.0
5         Midtown South   0.0
6   Morningside Heights   0.0
7           Murray Hill   0.0
8                  Noho   0.0
9      Roosevelt Is

                 venue  freq
0             Gramercy  0.18
1            Yorkville  0.09
2      Upper West Side  0.09
3         East Village  0.09
4        Carnegie Hill  0.05
5      Lower East Side  0.05
6         Sutton Place  0.05
7       Lincoln Square  0.05
8           Lenox Hill  0.05
9           Tudor City  0.05
10         Murray Hill  0.05
11   Greenwich Village  0.05
12  Financial District  0.05
13        Civic Center  0.05
14             Chelsea  0.05
15      Central Harlem  0.05
16     Upper East Side  0.00
17          Turtle Bay  0.00
18             Tribeca  0.00
19  Washington Heights  0.00


----Bakery----
                 venue  freq
0         Little Italy  0.08
1            Chinatown  0.07
2      Upper East Side  0.07
3              Chelsea  0.05
4   Washington Heights  0.05
5          East Harlem  0.05
6              Midtown  0.05
7        Midtown South  0.04
8         Civic Center  0.04
9      Upper West Side  0.04
10                Soho  0.04
11      Lincoln Square  0.

                  venue  freq
0                  Soho  0.21
1     Greenwich Village  0.12
2             Chinatown  0.08
3          West Village  0.08
4       Upper East Side  0.08
5               Tribeca  0.04
6               Midtown  0.04
7        Manhattanville  0.04
8                  Noho  0.04
9          Little Italy  0.04
10           Lenox Hill  0.04
11        Midtown South  0.04
12             Flatiron  0.04
13              Chelsea  0.04
14       Central Harlem  0.04
15  Morningside Heights  0.00
16           Tudor City  0.00
17         Sutton Place  0.00
18      Stuyvesant Town  0.00
19           Turtle Bay  0.00


----Boxing Gym----
                  venue  freq
0                  Soho  0.11
1               Midtown  0.11
2          Little Italy  0.11
3         Midtown South  0.11
4          Civic Center  0.11
5            Turtle Bay  0.11
6            Tudor City  0.11
7           Murray Hill  0.11
8                  Noho  0.11
9     Battery Park City  0.00
10  Morningside Hei

                  venue  freq
0                  Noho   0.4
1          West Village   0.2
2        Lincoln Square   0.2
3          Sutton Place   0.2
4     Battery Park City   0.0
5                  Soho   0.0
6               Midtown   0.0
7         Midtown South   0.0
8   Morningside Heights   0.0
9           Murray Hill   0.0
10     Roosevelt Island   0.0
11      Stuyvesant Town   0.0
12       Manhattanville   0.0
13              Tribeca   0.0
14           Tudor City   0.0
15           Turtle Bay   0.0
16      Upper East Side   0.0
17      Upper West Side   0.0
18   Washington Heights   0.0
19          Marble Hill   0.0


----Cantonese Restaurant----
                  venue  freq
0             Chinatown   1.0
1     Battery Park City   0.0
2       Stuyvesant Town   0.0
3               Midtown   0.0
4         Midtown South   0.0
5   Morningside Heights   0.0
6           Murray Hill   0.0
7                  Noho   0.0
8      Roosevelt Island   0.0
9                  Soho   0.0
10       

                  venue  freq
0               Chelsea   0.5
1        Lincoln Square   0.5
2     Battery Park City   0.0
3       Stuyvesant Town   0.0
4               Midtown   0.0
5         Midtown South   0.0
6   Morningside Heights   0.0
7           Murray Hill   0.0
8                  Noho   0.0
9      Roosevelt Island   0.0
10                 Soho   0.0
11         Sutton Place   0.0
12       Manhattanville   0.0
13              Tribeca   0.0
14           Tudor City   0.0
15           Turtle Bay   0.0
16      Upper East Side   0.0
17      Upper West Side   0.0
18   Washington Heights   0.0
19         West Village   0.0


----Comedy Club----
                  venue  freq
0              Gramercy  0.25
1     Greenwich Village  0.25
2               Clinton  0.25
3          Hudson Yards  0.25
4         Midtown South  0.00
5   Morningside Heights  0.00
6           Murray Hill  0.00
7                  Noho  0.00
8      Roosevelt Island  0.00
9                  Soho  0.00
10      Stuyvesant

...

                  venue  freq
0             Chinatown  0.12
1          Little Italy  0.12
2               Midtown  0.08
3              Flatiron  0.08
4           Murray Hill  0.08
5                  Soho  0.08
6            Lenox Hill  0.08
7          East Village  0.08
8         Carnegie Hill  0.04
9          Sutton Place  0.04
10            Yorkville  0.04
11              Chelsea  0.04
12       Central Harlem  0.04
13           Tudor City  0.04
14           Turtle Bay  0.00
15  Morningside Heights  0.00
16              Tribeca  0.00
17      Upper East Side  0.00
18      Stuyvesant Town  0.00
19      Upper West Side  0.00


----Sandwich Place----
                  venue  freq
0           Murray Hill  0.09
1               Clinton  0.09
2               Midtown  0.07
3          Civic Center  0.07
4          Little Italy  0.07
5                  Noho  0.07
6          West Village  0.04
7    Washington Heights  0.04
8             Chinatown  0.04
9           East Harlem  0.04
10   Financial 

----Soccer Field----
                  venue  freq
0             Yorkville   0.5
1      Roosevelt Island   0.5
2         Carnegie Hill   0.0
3           Marble Hill   0.0
4               Midtown   0.0
5         Midtown South   0.0
6   Morningside Heights   0.0
7           Murray Hill   0.0
8                  Noho   0.0
9                  Soho   0.0
10      Stuyvesant Town   0.0
11         Sutton Place   0.0
12              Tribeca   0.0
13           Tudor City   0.0
14           Turtle Bay   0.0
15      Upper East Side   0.0
16      Upper West Side   0.0
17   Washington Heights   0.0
18         West Village   0.0
19       Manhattanville   0.0


----Social Club----
                  venue  freq
0              Gramercy   1.0
1     Battery Park City   0.0
2        Manhattanville   0.0
3               Midtown   0.0
4         Midtown South   0.0
5   Morningside Heights   0.0
6           Murray Hill   0.0
7                  Noho   0.0
8      Roosevelt Island   0.0
9                  Soho   0

                  venue  freq
0          Civic Center   1.0
1     Battery Park City   0.0
2       Stuyvesant Town   0.0
3               Midtown   0.0
4         Midtown South   0.0
5   Morningside Heights   0.0
6           Murray Hill   0.0
7                  Noho   0.0
8      Roosevelt Island   0.0
9                  Soho   0.0
10         Sutton Place   0.0
11       Manhattanville   0.0
12              Tribeca   0.0
13           Tudor City   0.0
14           Turtle Bay   0.0
15      Upper East Side   0.0
16      Upper West Side   0.0
17   Washington Heights   0.0
18         West Village   0.0
19          Marble Hill   0.0


----Supermarket----
                  venue  freq
0    Washington Heights  0.15
1   Morningside Heights  0.08
2          Hudson Yards  0.08
3          West Village  0.08
4             Chinatown  0.08
5        Manhattanville  0.08
6               Clinton  0.08
7         Carnegie Hill  0.08
8          Little Italy  0.08
9      Roosevelt Island  0.08
10               I

                  venue  freq
0       Lower East Side   0.2
1   Morningside Heights   0.2
2            Turtle Bay   0.2
3            Tudor City   0.2
4          Sutton Place   0.2
5      Roosevelt Island   0.0
6           Marble Hill   0.0
7               Midtown   0.0
8         Midtown South   0.0
9           Murray Hill   0.0
10                 Noho   0.0
11    Battery Park City   0.0
12                 Soho   0.0
13       Manhattanville   0.0
14              Tribeca   0.0
15      Upper East Side   0.0
16      Upper West Side   0.0
17   Washington Heights   0.0
18         West Village   0.0
19      Stuyvesant Town   0.0


----Tennis Stadium----
                  venue  freq
0           Marble Hill   1.0
1     Battery Park City   0.0
2         Carnegie Hill   0.0
3               Midtown   0.0
4         Midtown South   0.0
5   Morningside Heights   0.0
6           Murray Hill   0.0
7                  Noho   0.0
8      Roosevelt Island   0.0
9                  Soho   0.0
10      Stuyves

                  venue  freq
0                  Noho   1.0
1     Battery Park City   0.0
2         Carnegie Hill   0.0
3           Marble Hill   0.0
4               Midtown   0.0
5         Midtown South   0.0
6   Morningside Heights   0.0
7           Murray Hill   0.0
8      Roosevelt Island   0.0
9                  Soho   0.0
10      Stuyvesant Town   0.0
11         Sutton Place   0.0
12              Tribeca   0.0
13           Tudor City   0.0
14           Turtle Bay   0.0
15      Upper East Side   0.0
16      Upper West Side   0.0
17   Washington Heights   0.0
18         West Village   0.0
19       Manhattanville   0.0


----Veterinarian----
                  venue  freq
0                Inwood   1.0
1     Battery Park City   0.0
2        Manhattanville   0.0
3               Midtown   0.0
4         Midtown South   0.0
5   Morningside Heights   0.0
6           Murray Hill   0.0
7                  Noho   0.0
8      Roosevelt Island   0.0
9                  Soho   0.0
10      Stuyvesan

# Scrolling through venues we find that Hamilton Heights neighborhood of Manhattan has the highest frequency of Pizza places

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Venue Category']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Neighborhood'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Neighborhood'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Venue Category'] = manhattan_grouped['Venue Category']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Venue Category,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood
0,Accessories Store,West Village,Washington Heights,Upper West Side,Financial District,Lenox Hill
1,Adult Boutique,Sutton Place,Yorkville,Financial District,Lenox Hill,Inwood
2,African Restaurant,Central Harlem,Yorkville,Flatiron,Lincoln Square,Lenox Hill
3,American Restaurant,West Village,Tribeca,Clinton,Chinatown,Chelsea
4,Antique Shop,Civic Center,Yorkville,Flatiron,Lincoln Square,Lenox Hill
5,Arepa Restaurant,East Village,Washington Heights,Financial District,Lenox Hill,Inwood
6,Argentinian Restaurant,Lower East Side,Tribeca,Noho,East Village,Carnegie Hill
7,Art Gallery,Chelsea,Noho,Lower East Side,Flatiron,Tribeca
8,Art Museum,Carnegie Hill,Upper East Side,Yorkville,Financial District,Lenox Hill
9,Arts & Crafts Store,East Village,Upper West Side,Manhattan Valley,Morningside Heights,Soho


In [34]:
address = 'Hamilton Heights, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.8241451 -73.9500618


# Now let's look for pizza places in Hamilton Heights

In [35]:
search_query = 'Pizza'
radius = 500
print(search_query + ' .... OK!')

Pizza .... OK!


In [36]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=W03DEFIYSYUW3GCTD0ZGYPKADF0Q1DDPJYL1BUQ4S4I2TIP3&client_secret=TF0ZKC1WEP2XAUH3TO2PYBTKEKSW5DNZV0LRZHTOW0I1UKF3&ll=40.8241451,-73.9500618&v=20200926&query=Pizza&radius=500&limit=100'

In [37]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f7089d486ebb970ea00f6d8'},
 'response': {'venues': [{'id': '59beea8a26659b702af8a8e0',
    'name': "Uncle Tony's Pizza",
    'location': {'address': '1596 Amsterdam Ave',
     'crossStreet': 'btwn W 139th & 138th St',
     'lat': 40.82144,
     'lng': -73.950766,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.82144,
       'lng': -73.950766},
      {'label': 'entrance', 'lat': 40.821378, 'lng': -73.950631}],
     'distance': 306,
     'postalCode': '10031',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1596 Amsterdam Ave (btwn W 139th & 138th St)',
      'New York, NY 10031',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'prim

In [38]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",471866,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/uncle-tonys-pizz...,False,59beea8a26659b702af8a8e0,1596 Amsterdam Ave,US,New York,United States,btwn W 139th & 138th St,306,"[1596 Amsterdam Ave (btwn W 139th & 138th St),...","[{'label': 'display', 'lat': 40.82144, 'lng': ...",40.821440,-73.950766,10031,NY,Uncle Tony's Pizza,v-1601210836,NaN
1,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,505380aee4b07ab78c6545fe,348 W 145th St,US,New York,United States,at St Nicholas Ave,492,"[348 W 145th St (at St Nicholas Ave), New York...","[{'label': 'display', 'lat': 40.82383014147579...",40.823830,-73.944236,10039,NY,Vittorio's Pizza Plus,v-1601210836,51678940
2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bd789145cf276b06a959b00,3587 Broadway,US,New York,United States,NaN,565,"[3587 Broadway, New York, NY 10031, United Sta...","[{'label': 'display', 'lat': 40.82910885707959...",40.829109,-73.948654,10031,NY,Domino's Pizza,v-1601210836,NaN
3,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4df94db2a809fe7742217cf6,1747 Amsterdam Ave,US,New York,United States,147th & 148th,366,"[1747 Amsterdam Ave (147th & 148th), New York,...","[{'label': 'display', 'lat': 40.826122, 'lng':...",40.826122,-73.946579,10031,NY,P&M Pizza,v-1601210836,NaN
4,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bd5cf1229eb9c74d5f893e1,394 W 145th St,US,New York,United States,145th snd St. Nicholas,433,"[394 W 145th St (145th snd St. Nicholas), New ...","[{'label': 'display', 'lat': 40.82411087887829...",40.824111,-73.944911,10031,NY,New Millennium Pizza,v-1601210836,NaN


In [39]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Uncle Tony's Pizza,Pizza Place,1596 Amsterdam Ave,US,New York,United States,btwn W 139th & 138th St,306,"[1596 Amsterdam Ave (btwn W 139th & 138th St),...","[{'label': 'display', 'lat': 40.82144, 'lng': ...",40.821440,-73.950766,10031,NY,59beea8a26659b702af8a8e0
1,Vittorio's Pizza Plus,Pizza Place,348 W 145th St,US,New York,United States,at St Nicholas Ave,492,"[348 W 145th St (at St Nicholas Ave), New York...","[{'label': 'display', 'lat': 40.82383014147579...",40.823830,-73.944236,10039,NY,505380aee4b07ab78c6545fe
2,Domino's Pizza,Pizza Place,3587 Broadway,US,New York,United States,NaN,565,"[3587 Broadway, New York, NY 10031, United Sta...","[{'label': 'display', 'lat': 40.82910885707959...",40.829109,-73.948654,10031,NY,4bd789145cf276b06a959b00
3,P&M Pizza,Pizza Place,1747 Amsterdam Ave,US,New York,United States,147th & 148th,366,"[1747 Amsterdam Ave (147th & 148th), New York,...","[{'label': 'display', 'lat': 40.826122, 'lng':...",40.826122,-73.946579,10031,NY,4df94db2a809fe7742217cf6
4,New Millennium Pizza,Pizza Place,394 W 145th St,US,New York,United States,145th snd St. Nicholas,433,"[394 W 145th St (145th snd St. Nicholas), New ...","[{'label': 'display', 'lat': 40.82411087887829...",40.824111,-73.944911,10031,NY,4bd5cf1229eb9c74d5f893e1
5,Papa John's Pizza,Pizza Place,1709 Amsterdam Ave,US,New York,United States,NaN,219,"[1709 Amsterdam Ave, New York, NY 10031, Unite...","[{'label': 'display', 'lat': 40.82487430556074...",40.824874,-73.947635,10031,NY,5812ea4538fa932fe0cc3951
6,Wholesome Garden Pizza,Pizza Place,3529,US,New York,United States,W145 & Broadway,224,"[3529 (W145 & Broadway), New York, NY 10031, U...","[{'label': 'display', 'lat': 40.82613920553293...",40.826139,-73.950439,10031,NY,5510a686498e3dbab70a132d
7,Domino's Pizza,Pizza Place,3624 Broadway,US,New York,United States,W 149th Street,566,"[3624 Broadway (W 149th Street), New York, NY ...","[{'label': 'display', 'lat': 40.829007, 'lng':...",40.829007,-73.948089,10031,NY,53e50a52498e020ca20ecb5f
8,Pizza Ghor,Pizza Place,3609 Broadway,US,New York,United States,149th & 148th Streets,522,"[3609 Broadway (149th & 148th Streets), New Yo...","[{'label': 'display', 'lat': 40.828779, 'lng':...",40.828779,-73.949062,10031,NY,5726570b498e4a0a3ebed95c
9,Pizza Guys,Pizza Place,3609 Broadway,US,New York,United States,149th Street,536,"[3609 Broadway (149th Street), New York, NY 10...","[{'label': 'display', 'lat': 40.82889175415039...",40.828892,-73.948990,10031,NY,52d336af498e3355ad7a4358


In [40]:
dataframe_filtered.name

0         Uncle Tony's Pizza
1      Vittorio's Pizza Plus
2             Domino's Pizza
3                  P&M Pizza
4       New Millennium Pizza
5          Papa John's Pizza
6     Wholesome Garden Pizza
7             Domino's Pizza
8                 Pizza Ghor
9                 Pizza Guys
10              Paul's Pizza
11               Jumbo Pizza
12     Harlem Brothers Pizza
13              Olga's Pizza
14                      Fumo
Name: name, dtype: object

# Let's map out these pizza places

In [41]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hamilton Heights',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Chosen at random, let's dive deeper into Fumo, see how it rates and what they do well

In [49]:
venue_id = '56d8e01d498ef1500ae7fbfe' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])
9.0


# This is a pretty good rating! Let's see about their tips and ratings.

In [50]:
result['response']['venue']['tips']['count']
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"The funghi pizza is so delicious. Fresh ingredients, not overly spiced and NOT OILY. I loved the meatballs in marinara sauce. Wasn't crazy about the garlic bread. The pizza though 👌🏻👌🏻👌🏻",2,0,59b36487829b0c469257497e,Heba-I-am,NaN,151113345


# Let's explore one more: Uncle Tony's

In [53]:
venue_id = '59beea8a26659b702af8a8e0' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])
8.2


In [54]:
result['response']['venue']['tips']['count']
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,Order the white pie with mushrooms and ricotta cheese. The ingredients were super fresh!,0,0,59e51d029e0d543217abe868,Joe,M,1814497


# That's also not bad! One more, how their neighborhood about Papa Johns?

In [57]:
venue_id = '5812ea4538fa932fe0cc3951' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])
6.5


In [58]:
result['response']['venue']['tips']['count']
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

IndexError: list index out of range

# Not good, and unfortunately we can't seem to retrieve their tips.

# Out of curiosity let's check another local pizza place: P&M

In [64]:
venue_id = '4df94db2a809fe7742217cf6' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])
6.0


In [65]:
result['response']['venue']['tips']['count']
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"LOVE the food here, especially since there aren't that many options in the area. Delivery is always quick.",0,0,541dcca6498e1208966c6b93,darnisha,b,192019


# Now this is interesting. While P&M seems to be rated lower than Papa Johns, we did find a positive review.

# We'll take a look at just one more pizza place: New Millennium Pizza

In [72]:
venue_id = '4bd5cf1229eb9c74d5f893e1' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])
This venue has not been rated yet.


In [73]:
result['response']['venue']['tips']['count']
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,Pretty decent slice. $2.00 for plain,0,0,5595bffb498eaaf2e5d1c3d3,Alex,F,1350822


# No ratings, unfortunately, but a review we can work with.

# Results

Our results showed us that Hamilton Heights in Manhattan appears to be a popular neighborhood for pizza places, with over a dozen locations of varying ratings.

Fumo rated highest at 9.0, followed by Uncle Tony's at 8.2. Papa John's rated 6.5 and P&M rated 6.0. New Millennium did not return a rating.

# Discussion

Exploring five locations, Fumo, Uncle Tony's, Papa Johns, P0&M, and New Millennium, we learned Fumo, has delicious, fresh pizza. Their funghi pizza might be a recipe worth looking into, however they did not have great garlic bread, so we should make ours better than their, knowing what not to do. We also discovered that reviews were positive for Uncle Tony's, specifically in their use a fresh ingredients (something noted in Fumo's review as well), which will definitely be something to keep in mind. The P&M reviewer noted quick delivery, and the New Millennium reviewer noted a reasonable price for a slice.

Reviewers generally seemed to prefer local pizza places to a chain pizza place like Papa Johns. Even a lower rated place like P&M had a positive review. Seemingly contradictorily, the reviewer for P&M implied there aren't many choices in the area, though they didn't specify if this meant food ingeneral. Perhaps they would benefit from an analysis like this one.

# Conclusions

For a chance at success at opening a pizza place in Manhattan, NY, you should focus your attention on the Hamilton Heights neighborhood and the locations found there. Getting the word out will be paramount so folks know that there are options available. Your menu should include unique recipes (mushrooms seem to be popular), and the pizza should be made with the freshenst ingredients. You should ensure quick delivery, and your price per slice should be competitive.